## 1) Let's start by getting a feel for the data.

In [118]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [119]:
df = pd.read_csv('../data/raw_car_data.csv')

In [120]:
display('Null Values Count', df.isnull().sum())
display('Shape', df.shape)
display('info', df.info())
display(df.head())

'Null Values Count'

Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  42
New_Price            5195
Price                   0
dtype: int64

'Shape'

(6019, 13)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 13 columns):
Name                 6019 non-null object
Location             6019 non-null object
Year                 6019 non-null int64
Kilometers_Driven    6019 non-null int64
Fuel_Type            6019 non-null object
Transmission         6019 non-null object
Owner_Type           6019 non-null object
Mileage              6017 non-null object
Engine               5983 non-null object
Power                5983 non-null object
Seats                5977 non-null float64
New_Price            824 non-null object
Price                6019 non-null float64
dtypes: float64(2), int64(2), object(9)
memory usage: 611.4+ KB


'info'

None

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


## 2) There are a lot of null values, let's clean that up.

In [121]:
#New_Price column, not sure what that means and it is ~5/6 null values, 
# dropping column. 
df = df.drop(columns=['New_Price'])

In [122]:
df1 = df[df.isna().any(axis=1)]
df1
df1.count()

Name                 44
Location             44
Year                 44
Kilometers_Driven    44
Fuel_Type            44
Transmission         44
Owner_Type           44
Mileage              42
Engine                8
Power                 8
Seats                 2
Price                44
dtype: int64

In [123]:
#Quick visual scan of the data shows null values often missing across 
# an individual row, so perhaps only 44 records affected, dropping rows.
df = df.dropna()

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5975 entries, 0 to 6018
Data columns (total 12 columns):
Name                 5975 non-null object
Location             5975 non-null object
Year                 5975 non-null int64
Kilometers_Driven    5975 non-null int64
Fuel_Type            5975 non-null object
Transmission         5975 non-null object
Owner_Type           5975 non-null object
Mileage              5975 non-null object
Engine               5975 non-null object
Power                5975 non-null object
Seats                5975 non-null float64
Price                5975 non-null float64
dtypes: float64(2), int64(2), object(8)
memory usage: 606.8+ KB


## 3) We have many 'object' columns which could be numbers instead, let's sort that out. 

In [125]:
#power (bhp), engine (cc), mileage (kmpl) should be numeric values
df.rename(columns={'Power': 'Power-bhp', 'Engine': 'Engine-cc', 'Mileage': 'Mileage-kmpl'}, inplace=True)

In [126]:
df['Power-bhp'] = df['Power-bhp'].str.replace('bhp', '')
df['Engine-cc'] = df['Engine-cc'].str.replace('cc', '')
df['Mileage-kmpl'] = df['Mileage-kmpl'].str.replace('kmpl', '')

In [127]:
df['Engine-cc'] = df['Engine-cc'].str.replace('CC', '')
df['Power-bhp'] = df['Power-bhp'].str.replace(' ', '')
df['Mileage-kmpl'] = df['Mileage-kmpl'].str.replace('km/kg', '')

In [128]:
df['Engine-cc'] = df['Engine-cc'].str.replace(' ', '')

In [129]:
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage-kmpl,Engine-cc,Power-bhp,Seats,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6,998,58.16,5.0,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.2,5.0,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2,1199,88.7,5.0,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7.0,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2,1968,140.8,5.0,17.74


In [130]:
df['Engine-cc'] = df['Engine-cc'].astype(int)

In [131]:
df['Mileage-kmpl'] = df['Mileage-kmpl'].astype(float)

In [132]:
#there are a few type written nulls in here that weren't caught earlier
#using errors='coerce' here will turn them into zeros, for now... 
df['Power-bhp'] = pd.to_numeric(df['Power-bhp'], errors='coerce')

In [133]:
#The column called "Owner Type" could be useful info, but it is also a string
#Let's change this column name too, so we don't forget what it is! 
df = df.rename(columns={'Owner_Type': 'Number_Owners'})

In [134]:
df.Number_Owners.unique()

array(['First', 'Second', 'Fourth & Above', 'Third'], dtype=object)

In [135]:
df['Number_Owners'] = df['Number_Owners'].replace({'First': 1, 'Second': 2, 'Third': 3, 'Fourth & Above': 4})

In [136]:
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Number_Owners,Mileage-kmpl,Engine-cc,Power-bhp,Seats,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,1,26.60,998,58.16,5.0,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,1,19.67,1582,126.20,5.0,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,1,18.20,1199,88.70,5.0,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,1,20.77,1248,88.76,7.0,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,2,15.20,1968,140.80,5.0,17.74


In [137]:
# Transmission only has binary options, let's get that into number form too. 
df = df.rename(columns={'Transmission': 'Is_Manual'})
df['Is_Manual'] = df['Is_Manual'].replace({'Manual': 1, 'Automatic': 0})
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Is_Manual,Number_Owners,Mileage-kmpl,Engine-cc,Power-bhp,Seats,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,1,1,26.60,998,58.16,5.0,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,1,1,19.67,1582,126.20,5.0,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,1,1,18.20,1199,88.70,5.0,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,1,1,20.77,1248,88.76,7.0,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,0,2,15.20,1968,140.80,5.0,17.74


In [144]:
df.Fuel_Type.unique()

array(['CNG', 'Diesel', 'Petrol', 'LPG'], dtype=object)

In [ ]:
# what is a CNG or LPG fuel type??? #Annoying ... *googling* ...
# CNG = Compressed Natural Gas and LPG is either Propane or Butane!!! 
# how big of a factor is this???

In [145]:
df.Fuel_Type.value_counts()

Diesel    3195
Petrol    2714
CNG         56
LPG         10
Name: Fuel_Type, dtype: int64

In [146]:
#Only 66 of my ~6000 vales, I'm going to cut these to keep things simple
#Then, turn this column into a binary as well. 
df = df.rename(columns={'Fuel_Type': 'Is_Diesel'})
df = df[df.Is_Diesel != 'CNG']
df = df[df.Is_Diesel != 'LPG']
df['Is_Diesel'] = df['Is_Diesel'].replace({'Diesel': 1, 'Petrol': 0})
df.head()

,Name,Location,Year,Kilometers_Driven,Is_Diesel,Is_Manual,Number_Owners,Mileage-kmpl,Engine-cc,Power-bhp,Seats,Price
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,1,1,1,19.67,1582,126.20,5.0,12.50
2,Honda Jazz V,Chennai,2011,46000,0,1,1,18.20,1199,88.70,5.0,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,1,1,1,20.77,1248,88.76,7.0,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,1,0,2,15.20,1968,140.80,5.0,17.74
6,Nissan Micra Diesel XV,Jaipur,2013,86999,1,1,1,23.08,1461,63.10,5.0,3.50


In [148]:
df.describe()

,Year,Kilometers_Driven,Is_Diesel,Is_Manual,Number_Owners,Mileage-kmpl,Engine-cc,Power-bhp,Seats,Price
count,5909.000000,5.909000e+03,5909.000000,5909.000000,5909.000000,5909.000000,5909.000000,5807.000000,5909.000000,5909.000000
mean,2013.385344,5.870272e+04,0.540701,0.710780,1.200542,18.108766,1627.640379,113.827634,5.281604,9.570237
std,3.252403,9.203877e+04,0.498383,0.453439,0.453557,4.449447,601.259874,53.903495,0.810749,11.248288
min,1998.000000,1.710000e+02,0.000000,0.000000,1.000000,0.000000,624.000000,34.200000,0.000000,0.440000
25%,2011.000000,3.385500e+04,0.000000,0.000000,1.000000,15.170000,1198.000000,78.000000,5.000000,3.500000
50%,2014.000000,5.300000e+04,1.000000,1.000000,1.000000,18.150000,1495.000000,98.600000,5.000000,5.700000
75%,2016.000000,7.300000e+04,1.000000,1.000000,1.000000,21.100000,1991.000000,139.040000,5.000000,10.000000
max,2019.000000,6.500000e+06,1.000000,1.000000,4.000000,28.400000,5998.000000,560.000000,10.000000,160.000000


In [151]:
#Lastly, seats as a float seems strange, let's turn that to an int.
df.Seats = df.Seats.astype(int)

In [152]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5909 entries, 1 to 6018
Data columns (total 12 columns):
Name                 5909 non-null object
Location             5909 non-null object
Year                 5909 non-null int64
Kilometers_Driven    5909 non-null int64
Is_Diesel            5909 non-null int64
Is_Manual            5909 non-null int64
Number_Owners        5909 non-null int64
Mileage-kmpl         5909 non-null float64
Engine-cc            5909 non-null int64
Power-bhp            5807 non-null float64
Seats                5909 non-null int64
Price                5909 non-null float64
dtypes: float64(3), int64(7), object(2)
memory usage: 600.1+ KB


In [155]:
# Much BETTER!! Now let's save this cleaned up df as a new csv 
df.to_csv('../data/cars.csv', index=False)

## 4) It's time to [PLOT!!](https://github.com/JillCarrie/Mod4/tree/master/python_files/EDA_Visuals.ipynb)
This file is getting a bit LONG, so click the link to follow along.